# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey
# config gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Read the mouse data and the study results
weather_df = pd.read_csv('../output_data/cities.csv') 
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,yellowknife,1,CA,1612722042,66,62.4560,-114.3525,-40.00,5.75
1,1,wuxue,100,CN,1612722305,90,29.8506,115.5525,51.66,3.22
2,2,shamkhal,90,RU,1612722306,80,43.0596,47.3373,32.00,17.90
3,3,vaini,75,TO,1612722306,89,-21.2000,-175.2000,78.80,12.66
4,4,esperance,75,AU,1612722307,88,-33.8667,121.9000,62.60,11.50
...,...,...,...,...,...,...,...,...,...,...
521,521,abu kamal,0,SY,1612722449,60,34.4506,40.9171,59.36,7.65
522,522,turka,100,UA,1612722449,86,49.1541,23.0297,32.00,3.98
523,523,shagamu,86,NG,1612722449,60,6.8432,3.6478,86.00,7.78
524,524,alcaniz,67,ES,1612722210,54,41.0500,-0.1333,51.01,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations=weather_df[["Lat","Lng"]]
weight=weather_df.Humidity

In [9]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100
                                 ,point_radius=1)  # point radius how big shape around 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#narrow down weather cities df to cities that have the right vacation weather.  use 3 criteria to narrow cities down to ~10 choices
vacay_humidity_limit = 23
vacay_temp_max = 80
vacay_temp_min = 20
vacay_df = weather_df.loc[(weather_df.Humidity<vacay_humidity_limit)&(weather_df.Max_Temp<=vacay_temp_max)&(weather_df.Max_Temp>vacay_temp_min)]
#no need to drop null rows as that was done when building the weather df orifinally.
vacay_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
20,20,sabha,93,LY,1612722194,15,27.0377,14.4283,74.79,15.57
172,172,tombouctou,55,ML,1612722352,21,20.0000,-3.0000,70.59,12.08
185,185,biltine,5,TD,1612722356,16,14.5333,20.9167,75.00,11.01
209,209,englewood,20,US,1612722363,12,39.6478,-104.9878,53.01,21.85
274,274,mizdah,99,LY,1612722378,22,31.4451,12.9801,67.95,13.58
328,328,bozoum,45,CF,1612722391,19,6.3193,16.3799,78.28,4.36
336,336,sonoita,0,MX,1612722394,19,31.8500,-112.8333,70.00,1.14
352,352,kharan,0,PK,1612722398,20,28.5833,65.4167,62.15,6.82
382,382,taoudenni,0,ML,1612722412,22,22.6783,-3.9836,69.67,8.93
440,440,atar,0,MR,1612722428,21,20.5169,-13.0499,73.15,15.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_df
hotel_df["Hotel Name"]=""
hotel_df

C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
20,20,sabha,93,LY,1612722194,15,27.0377,14.4283,74.79,15.57,
172,172,tombouctou,55,ML,1612722352,21,20.0000,-3.0000,70.59,12.08,
185,185,biltine,5,TD,1612722356,16,14.5333,20.9167,75.00,11.01,
209,209,englewood,20,US,1612722363,12,39.6478,-104.9878,53.01,21.85,
274,274,mizdah,99,LY,1612722378,22,31.4451,12.9801,67.95,13.58,
328,328,bozoum,45,CF,1612722391,19,6.3193,16.3799,78.28,4.36,
336,336,sonoita,0,MX,1612722394,19,31.8500,-112.8333,70.00,1.14,
352,352,kharan,0,PK,1612722398,20,28.5833,65.4167,62.15,6.82,
382,382,taoudenni,0,ML,1612722412,22,22.6783,-3.9836,69.67,8.93,
440,440,atar,0,MR,1612722428,21,20.5169,-13.0499,73.15,15.12,


In [17]:
params = {"keyword":"hotel",
          "radius": 5000,
          "type": "lodging",
          "key": gkey    
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in hotel_df.iterrows():
    if row["City_ID"]==172:
        break   #test break to just call api 1x
    lat = row["Lat"]
    lng = row["Lng"]
    print(row)
    params["location"]= f"{lat},{lng}"
    hotel_resp= requests.get(base_url, params=params).json()
    hotel_df.loc[index,"Hotel Name"]=hotel_resp["results"][0]["name"]
    #print(json.dumps(hotel_resp["results"], indent=4))
    #possible functionality upgrade for next iteration: add loop through response to check business_status == "OPERATIONAL"
    
hotel_df    

City_ID               20
City               sabha
Cloudiness            93
Country               LY
Date          1612722194
Humidity              15
Lat              27.0377
Lng              14.4283
Max_Temp           74.79
Wind_Speed         15.57
Hotel Name              
Name: 20, dtype: object


C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
20,20,sabha,93,LY,1612722194,15,27.0377,14.4283,74.79,15.57,فندق علي
172,172,tombouctou,55,ML,1612722352,21,20.0000,-3.0000,70.59,12.08,
185,185,biltine,5,TD,1612722356,16,14.5333,20.9167,75.00,11.01,
209,209,englewood,20,US,1612722363,12,39.6478,-104.9878,53.01,21.85,
274,274,mizdah,99,LY,1612722378,22,31.4451,12.9801,67.95,13.58,
328,328,bozoum,45,CF,1612722391,19,6.3193,16.3799,78.28,4.36,
336,336,sonoita,0,MX,1612722394,19,31.8500,-112.8333,70.00,1.14,
352,352,kharan,0,PK,1612722398,20,28.5833,65.4167,62.15,6.82,
382,382,taoudenni,0,ML,1612722412,22,22.6783,-3.9836,69.67,8.93,
440,440,atar,0,MR,1612722428,21,20.5169,-13.0499,73.15,15.12,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']] 

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))